In [ ]:
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from collections import OrderedDict
import matplotlib.colors as colors
import matplotlib.cm as cmx
import pickle

from astropy.cosmology import wCDM
from astropy.cosmology import FlatLambdaCDM, FlatwCDM

In [ ]:
# get correct cosmology

h0 = 70
cosmo = wCDM(H0=h0, Om0=0.3, w0=-1, Ode0=0.7)

xaxis = np.arange(0.001,1.5,0.01)
theor_dist = np.array([cosmo.distmod(z).value for z in xaxis])

In [ ]:
# read SALT2mu results
cases_ddf = ['perfect3000', 'random3000', 'fiducial3000', '99SNIa1SNII', '99SNIa1SNIbc', 
             '99.4SNIa0.6CART', '99.9SNIa0.1SLSN']

data_ddf = {}

for name in cases_ddf:
    fname = '/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data3/DDF/results/v0/3000/fitres/'+ \
            'test_salt2mu_lowz_withbias_' + name + '.fitres'

    data_temp = pd.read_csv(fname, delim_whitespace=True, comment='#')
    
    if 'perfect' not in name:
        fitres_temp = pd.read_csv(fname, delim_whitespace=True, comment='#')
        print(name, ' fitres_temp.size = ', fitres_temp.shape[0])
        
        # 11 are Ia and 1 are lowz, different sims
        mask = np.logical_and(fitres_temp['SIM_TYPE_INDEX'].values != 11,
                              fitres_temp['SIM_TYPE_INDEX'].values != 1)
        print(name, ' mask.size = ', sum(mask))
        fitres_temp3 = fitres_temp[mask]

        # remove duplicated redshift
        z_all = []
        for j in range(fitres_temp3.shape[0]):
    
            z = fitres_temp3.iloc[j]['SIM_ZCMB']
            z_new = z
            if z in z_all:
                while z_new in z_all:
                    z_new = z + np.random.normal(loc=0, scale=0.001)
            
            fitres_temp3.loc[j, 'SIM_ZCMB'] = z_new
            z_all.append(z_new)
        
        indx = np.argsort(z_all)
        fitres_temp2 = fitres_temp3[['SIM_ZCMB', 'MU', 'MUERR', 'MURES']].iloc[indx]
        data_ddf[name] = fitres_temp2
        print(name, data_ddf[name].shape[0])
    else:
        data_ddf[name] = data_temp
        print(name, data_ddf[name].shape[0])

In [ ]:
all_shapes = {'SNIa-91bg': 'o',
              'SNIax': 's',
              'SNII': 'd',
              'SNIbc': 'X',
              'SLSN-I': 'v',
              'AGN': '^',
              'TDE': '<',
              'KN': '>',
              'CART': 'v'}

def make_remap_dict(file_extension):
    if 'wfd' == file_extension:
        remap_dict = OrderedDict({
                              'perfect3000': 'Perfect', 
                              'fiducial3000': 'Fiducial', 
                              'random3000': 'Random',
                              '72SNIa28SNII': 'SN-II 28',
                              '75SNIa25SNII': 'SN-II 25', 
                              '90SNIa10SNII': 'SN-II 10',
                              '95SNIa5SNII': 'SN-II 5',
                              '98SNIa2SNII': 'SN-II 2',
                              '99SNIa1SNII': 'SN-II 1',
                              '90SNIa10SNIbc': 'SN-Ibc 10',
                              '95SNIa5SNIbc': 'SN-Ibc 5',
                              '98SNIa2SNIbc': 'SN-Ibc 2',
                              '99SNIa1SNIbc': 'SN-Ibc 1',
                              '75SNIa25SNIax': 'SN-Iax 25',
                              '90SNIa10SNIax': 'SN-Iax 10',
                              '95SNIa5SNIax': 'SN-Iax 5',
                              '98SNIa2SNIax': 'SN-Iax 2',
                              '99SNIa1SNIax': 'SN-Iax 1',
                              '95SNIa5SNIa-91bg': 'SN-Ia-91bg 5',
                              '98SNIa2SNIa-91bg': 'SN-Ia-91bg 2',
                              '99SNIa1SNIa-91bg': 'SN-Ia-91bg 1',
                              '98SNIa2AGN': 'AGN 2',
                              '99SNIa1AGN': 'AGN 1',
                              '99SNIa1CART': 'CART 1'
                  })
    else:
        remap_dict = OrderedDict({
                          'perfect3000': 'Perfect', 
                          'fiducial3000': 'Fiducial', 
                          'random3000': 'Random',
                          '72SNIa28SNII': 'SN-II 28',
                          '75SNIa25SNII': 'SN-II 25', 
                          '90SNIa10SNII': 'SN-II 10',
                          '95SNIa5SNII': 'SN-II 5',
                          '98SNIa2SNII': 'SN-II 2',
                          '99SNIa1SNII': 'SN-II 1',
                          '95SNIa5SNIbc': 'SN-Ibc 5',
                          '98SNIa2SNIbc': 'SN-Ibc 2',
                          '99SNIa1SNIbc': 'SN-Ibc 1',
                          '90SNIa10SNIax': 'SN-Iax 10',
                          '95SNIa5SNIax': 'SN-Iax 5',
                          '98SNIa2SNIax': 'SN-Iax 2',
                          '99SNIa1SNIax': 'SN-Iax 1',
                           '99.4SNIa0.6CART': 'CART 0.6',
                          '99.9SNIa0.1SLSN': 'SLSN 0.1'
              })
    return(remap_dict)

In [ ]:
# read SALT2mu results
cases_wfd = ['perfect3000', 'random3000', 'fiducial3000', '99SNIa1SNII', '99SNIa1SNIbc', 
         '99SNIa1SNIax', '99SNIa1CART', '99SNIa1SNIa-91bg', '99SNIa1AGN']

data_wfd = {}

for name in cases_wfd:
    fname = '/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data3/WFD/results/v0/3000/fitres/'+ \
            'test_salt2mu_lowz_withbias_' + name + '.fitres'

    data_temp = pd.read_csv(fname, delim_whitespace=True, comment='#')
    
    if 'perfect' not in name:
        fitres_temp = pd.read_csv(fname, delim_whitespace=True, comment='#')
        
        # 11 are Ia and 1 are lowz, different sims
        mask = np.logical_and(fitres_temp['SIM_TYPE_INDEX'].values != 11,
                              fitres_temp['SIM_TYPE_INDEX'].values != 1)
        fitres_temp3 = fitres_temp[mask]

        # remove duplicated redshift
        z_all = []
        for j in range(fitres_temp3.shape[0]):
    
            z = fitres_temp3.iloc[j]['SIM_ZCMB']
            z_new = z
            if z in z_all:
                while z_new in z_all:
                    z_new = z + np.random.normal(loc=0, scale=0.001)
            
            fitres_temp3.at[j, 'SIM_ZCMB'] = z_new
            z_all.append(z_new)
        
        indx = np.argsort(z_all)
        fitres_temp2 = fitres_temp3[['SIM_ZCMB', 'MU', 'MUERR','MURES']].iloc[indx]
        data_wfd[name] = fitres_temp2
        print(name, data_wfd[name].shape[0])
    else:
        data_wfd[name] = data_temp
        print(name, data_wfd[name].shape[0])


In [ ]:
# Color map
rainbow = cm = plt.get_cmap('plasma_r')
cNorm  = colors.LogNorm(vmin=1, vmax=52) #colors.Normalize(vmin=0, vmax=50)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=rainbow)
color_map = scalarMap.to_rgba(np.arange(1, 52))

In [ ]:
a_file = open("../../plots_for_paper/colors.pkl", "rb")

contaminant_colors = pickle.load(a_file)

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(16,8), gridspec_kw={'height_ratios': [3, 1]})


for i in [0,1]:
    axs[1][i].set_ylabel(r'$\Delta \mu$', fontsize=16)
    axs[1][i].set_xlabel(r'$z$', fontsize=16)
    
    axs[0][i].set_ylabel(r'$\mu$', fontsize=16)
    axs[0][i].set_xticks([])
    
    axs[1][i].set_xlim(0, 1.5)
    axs[1][i].set_xticks(np.arange(0,1.5,0.2))
    
    
    if i == 0:
        axs[0][i].plot(xaxis, theor_dist, lw=2, color='black', label=r'$\Lambda$CDM', zorder=2)
        axs[0][i].errorbar(data_ddf['perfect3000']['SIM_ZCMB'], data_ddf['perfect3000']['MU'],
             yerr=data_ddf['perfect3000']['MUERR'], fmt ='*', color='gray', alpha=0.075,
             label='Ia', zorder=1)
        
        axs[1][i].errorbar(data_ddf['perfect3000']['SIM_ZCMB'], 
              data_ddf['perfect3000']['MU'] - cosmo.distmod(data_ddf['perfect3000']['SIM_ZCMB']).value,
             yerr=data_ddf['perfect3000']['MUERR'], fmt ='*', color='gray', alpha=0.075,
             label='Ia', zorder=1)

        for k in range(3, len(cases_ddf)):
            for key in all_shapes:
                if key in cases_wfd[i]:
                    cont = key
            
            axs[0][i].scatter(data_ddf[cases_ddf[k]]['SIM_ZCMB'], data_ddf[cases_ddf[k]]['MU'],
                marker=all_shapes[cont], s=40, 
                 color=contaminant_colors['ddf'][cases_ddf[k]], label=remap_dict_ddf[cases_ddf[k]], alpha=0.5, zorder=3)
    
            axs[0][i].errorbar(data_ddf[cases_ddf[k]]['SIM_ZCMB'], data_ddf[cases_ddf[k]]['MU'], 
                 yerr=data_ddf[cases_ddf[k]]['MUERR'], fmt=' ',
                 color=contaminant_colors['ddf'][cases_ddf[k]],  alpha=0.85,zorder=3)
    
            axs[1][i].scatter(data_ddf[cases_ddf[k]]['SIM_ZCMB'], 
                              data_ddf[cases_ddf[k]]['MU'] - cosmo.distmod(data_ddf[cases_ddf[k]]['SIM_ZCMB']).value,
                              marker=all_shapes[cont], s=40, 
                              color=contaminant_colors['ddf'][cases_ddf[k]], label=remap_dict_ddf[cases_ddf[k]], alpha=0.5)
    
            axs[1][i].errorbar(data_ddf[cases_ddf[k]]['SIM_ZCMB'], 
                               data_ddf[cases_ddf[k]]['MU'] - cosmo.distmod(data_ddf[cases_ddf[k]]['SIM_ZCMB']).value, 
                               yerr=data_ddf[cases_ddf[k]]['MUERR'], fmt=' ',
                               color=contaminant_colors['ddf'][cases_ddf[k]],  alpha=0.85)
        axs[0][i].legend(loc='lower right', title='DDF', frameon=False, fontsize=13)
        
    else:    
        axs[0][i].errorbar(data_wfd['perfect3000']['SIM_ZCMB'], data_wfd['perfect3000']['MU'],
                           yerr=data_wfd['perfect3000']['MUERR'], fmt ='*', color='gray', alpha=0.075,
                           label='Ia', zorder=1)
        
        axs[1][i].errorbar(data_wfd['perfect3000']['SIM_ZCMB'], 
                           data_wfd['perfect3000']['MU'] - cosmo.distmod(data_wfd['perfect3000']['SIM_ZCMB']).value,
                           yerr=data_wfd['perfect3000']['MUERR'], fmt ='*', color='gray', alpha=0.075,
                           label='Ia', zorder=1)

        for k in range(3, len(cases_wfd)):            
            for key in all_shapes:
                if key in cases_wfd[i]:
                    cont = key
            
            axs[0][i].scatter(data_wfd[cases_wfd[k]]['SIM_ZCMB'], data_wfd[cases_wfd[k]]['MU'],
                              marker=all_shapes[cont], s=40, 
                              color=contaminant_colors['wfd'][cases_wfd[k]], label=remap_dict_wfd[cases_wfd[k]], alpha=0.5)
    
            axs[0][i].errorbar(data_wfd[cases_wfd[k]]['SIM_ZCMB'], data_wfd[cases_wfd[k]]['MU'], 
                               yerr=data_wfd[cases_wfd[k]]['MUERR'], fmt=' ',
                               color=contaminant_colors['wfd'][cases_wfd[k]],  alpha=0.85)
        
            axs[1][i].scatter(data_wfd[cases_wfd[k]]['SIM_ZCMB'], 
                         data_wfd[cases_wfd[k]]['MU'] - cosmo.distmod(data_wfd[cases_wfd[k]]['SIM_ZCMB']).value,
                         marker=all_shapes[cont], s=40, 
                         color=contaminant_colors['wfd'][cases_wfd[k]], label=remap_dict_wfd[cases_wfd[i]], alpha=0.5)
    
            axs[1][i].errorbar(data_wfd[cases_wfd[k]]['SIM_ZCMB'], 
                          data_wfd[cases_wfd[k]]['MU'] - cosmo.distmod(data_wfd[cases_wfd[k]]['SIM_ZCMB']).value, 
                          yerr=data_wfd[cases_wfd[k]]['MUERR'], fmt=' ',
                          color=contaminant_colors['wfd'][cases_wfd[k]],  alpha=0.85)
        
        axs[0][i].plot(xaxis, theor_dist, lw=2, color='black', label=r'$\Lambda$CDM', zorder=2)
        axs[0][i].legend(loc='lower right', title='WFD', frameon=False, fontsize=13)
    
    
    axs[1][i].plot([0, 1.5], [0,0], lw=2, color='black')
    plt.setp(axs[0][i].get_legend().get_title(), fontsize=16)
    
    for j in range(2):
        axs[i][j].set_xlim(0, 1.5)
        
plt.subplots_adjust(hspace=0)
plt.tight_layout()
plt.savefig('hubble.png')